In [25]:
#Create database engine
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

#print tables in database
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


In [26]:
import pandas as pd

con = engine.connect()
rs = con.execute('SELECT * FROM Bookings')
df = pd.DataFrame(rs.fetchall())
df.columns = rs.keys()
con.close()
print(df.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [42]:
bookings = pd.read_sql_query('SELECT * FROM Bookings', engine)
print(bookings.head(3))

facilities = pd.read_sql_query('SELECT * FROM Facilities', engine)
print(facilities.head(3))

members = pd.read_sql_query('SELECT * FROM Members', engine)
print(members.head(3))


   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
   memid surname firstname                       address  zipcode  \
0      0   GUEST     GUEST                         GUEST        0   
1      1   Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2   Smith     Tracy  8 Bloomsbury Close, New York     4321   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07

In [28]:
#Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do.
query1 = 'SELECT DISTINCT name FROM Facilities WHERE membercost > 0 OR membercost IS NOT NULL' 

answer1 = pd.read_sql_query(query1, engine)
print(answer1)

              name
0   Tennis Court 1
1   Tennis Court 2
2  Badminton Court
3     Table Tennis
4   Massage Room 1
5   Massage Room 2
6     Squash Court
7    Snooker Table
8       Pool Table


In [29]:
#Q2: How many facilities do not charge a fee to members?

query2 = 'SELECT DISTINCT name FROM Facilities WHERE membercost = 0 OR membercost IS NULL' 
answer2 = pd.read_sql_query(query2, engine)
print(answer2)

              name
0  Badminton Court
1     Table Tennis
2    Snooker Table
3       Pool Table


In [31]:
#Q3: Write an SQL query to show a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost.
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question.

query3 = 'SELECT facid, name, membercost, monthlymaintenance FROM Facilities WHERE membercost < monthlymaintenance * 0.2 OR membercost IS NOT NULL ' 
answer3 = pd.read_sql_query(query3, engine)
print(answer3)

   facid             name  membercost  monthlymaintenance
0      0   Tennis Court 1         5.0                 200
1      1   Tennis Court 2         5.0                 200
2      2  Badminton Court         0.0                  50
3      3     Table Tennis         0.0                  10
4      4   Massage Room 1         9.9                3000
5      5   Massage Room 2         9.9                3000
6      6     Squash Court         3.5                  80
7      7    Snooker Table         0.0                  15
8      8       Pool Table         0.0                  15


In [34]:
#Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
#Try writing the query without using the OR operator.
query4 = 'SELECT * FROM Facilities WHERE facid = 1 UNION SELECT * FROM Facilities WHERE facid = 5'
answer4 = pd.read_sql_query(query4, engine)
print(answer4)

   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


In [37]:
#Q5: Produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100. Return the name and monthly maintenance of the facilities
#in question.

query5 = "SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS value FROM Facilities "
answer5 = pd.read_sql_query(query5, engine)
print(answer5)

              name  monthlymaintenance      value
0   Tennis Court 1                 200  expensive
1   Tennis Court 2                 200  expensive
2  Badminton Court                  50      cheap
3     Table Tennis                  10      cheap
4   Massage Room 1                3000  expensive
5   Massage Room 2                3000  expensive
6     Squash Court                  80      cheap
7    Snooker Table                  15      cheap
8       Pool Table                  15      cheap


In [40]:
#Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Try not to use the LIMIT clause for your solution.

query6 = "SELECT firstname, surname FROM Members ORDER BY joindate DESC"
answer6 = pd.read_sql_query(query6, engine)
print(answer6)

    firstname            surname
0      Darren              Smith
1       Erica            Crumpet
2        John               Hunt
3    Hyacinth         Tupperware
4   Millicent            Purview
5       Henry  Worthington-Smyth
6       David            Farrell
7   Henrietta             Rumney
8     Douglas              Jones
9   Ramnaresh             Sarwin
10       Joan             Coplin
11       Anna          Mackenzie
12    Matthew            Genting
13      David             Pinker
14    Timothy              Baker
15   Florence              Bader
16       Jack              Smith
17     Jemima            Farrell
18       Anne              Baker
19      David              Jones
20    Charles               Owen
21     Ponder           Stibbons
22        Tim             Boothe
23      Nancy               Dare
24     Burton              Tracy
25     Gerald            Butters
26     Janice           Joplette
27        Tim             Rownam
28      Tracy              Smith
29     Dar

In [ ]:
#Q7: Produce a list of all members who have used a tennis court.
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name.

WITH query AS 
	(SELECT DISTINCT memid, facid 
     FROM country_club.Bookings 
     WHERE facid = 0 OR facid = 1)



SELECT f.name, CONCAT(m.firstname,m.surname)
FROM country_club.Members AS m
--WHERE memid IN (query)
INNER JOIN country_club.query ON m.memid = query.memid
LEFT JOIN country_club.Facilities AS f ON query.facid = f.facid;

In [ ]:
#Q8: Produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30. Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries.